# Whole-genome analysis workflow

In [31]:
# ~2 minutes to install 
#%pip install -U --no-cache-dir scikit-learn scikit-optimize prefect prefect-ray ray plotly openpyxl shap lion_pytorch pytorch_tabnet xgboost neptune pyspark pyarrow dill fastnumbers

In [5]:
from prefect import task, flow
from prefect.task_runners import ConcurrentTaskRunner
from prefect_ray.task_runners import RayTaskRunner
from DillSerializer import DillSerializer
import ray

import pandas as pd
import numpy as np

import logging

!export PREFECT_LOGGING_LEVEL="WARNING"
ray.shutdown()
parallelRunner = ray.init(
  configure_logging=True,
  logging_level=logging.ERROR,
)
parallelRunner

Python version:,3.10.10
Ray version:,2.3.1


In [6]:
from sklearn.ensemble import (
    AdaBoostClassifier,
    RandomForestClassifier,
)
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC, SVC
from xgboost import XGBClassifier
from pytorch_tabnet.tab_model import TabNetClassifier
from lion_pytorch import Lion

from skopt.space import Categorical, Integer, Real

from env import neptune_api_token

RadialBasisSVC = SVC
RadialBasisSVC.__name__ = "RadialBasisSVC"

clearHistory = True
config = {
  'vcfLike': {  
    'path': 'Variant_report_NUPs_fixed_2022-03-28.xlsx',             # variant call table with annotations
    'sheet': "all cases vs all controls",                             # sheet name if Excel spreadsheet
    'indexColumn': ['chrom', 'position', 'Gene'],       # header that indexes variants (set as list with multiple columns)
    'binarize': True,                           # binarize variants to 0/1, or sum to weigh allele frequency
    'minAlleleFrequency': 0.05,           # filter out variants with allele frequency less than this
  # 'alleleModel': ['dominant', 'recessive', 'overDominant'],  # biallelic allele models to test on gene sets
    'filters': {
      } 
  }, # TODO handle genotypes from related individuals
  
  'geneSets' : { # TODO gene sets
      },
  
  'tracking': {
    'name': 'Nucleoporin genes, well-classified cases', # name of the experiment
    'entity': 'ejmockler',
    'project': 'ALS-NUPs-WellClassified',
    'plotAllSampleImportances': True,  # plot all sample feature importances in Neptune
    'token': neptune_api_token
  },

  'clinicalTable': {
      'path': 'ACWM.xlsx',                      # clinical data as Excel spreadsheet
      'idColumn': 'ExternalSampleId',           # genotype ID header
      'uniqueIdColumn': 'ExternalSubjectId',    # unique ID for each patient
      'labelColumn': 'Subject Group',                # header that has case/control labels
      'controlLabels': ['Non-Neurological Control'], # these labels include external sample IDs (like 1000 Genomes)
      'caseLabels': [],
      'controlAlias': 'control',
      'caseAlias': 'case',
      'filters': 'pct_european>=0.85',             # filter out nonhomogenous samples with less than 85% European ancestry
  },

  'externalTables': {
      'path': ['igsr-1000 genomes phase 3 release.tsv', 'accurateCases.csv'],  # external sample table
      'label': ['control', 'case'], # case | control | mixed (mixed labels are held out as an external test set)
      'idColumn': ['Sample name', 'id'],                        # sample ID header
      'filters': ["`Superpopulation code`=='EUR' & `Population name`!='Finnish'", ""], # remove finnish samples due to unusual homogeneity (verify w/ PCA)
  },

  'sampling': {
    'bootstrapIterations': 60, 
    'crossValIterations': 10,   # number of validations per bootstrap iteration
    'holdoutSplit': 0.1
  },

  'modelStack': {
    LinearSVC(): {
          "tol": Real(1e-6, 1e+1, prior="log-uniform"),
          "C": Real(1e-4, 1e+1, prior="log-uniform"),
      },
    RadialBasisSVC(probability=True, kernel="rbf"): {
        "tol": Real(1e-4, 1e+1, prior="log-uniform"),
        "C": Real(1e-4, 1e+1, prior="log-uniform"),
        "gamma": Categorical(["scale", "auto"]),
    },
    LogisticRegression(penalty="l2", solver="saga"): {
          "tol": Real(1e-6, 1e+1, prior="log-uniform"),
          "C": Real(1e-4, 1e+1, prior="log-uniform"),
      },
      # TabNetClassifier: {
      #     "n_d": Integer(8, 64),
      #     "n_a": Integer(8, 64),
      #     "n_steps": Integer(3, 10),
      #     "lambda_sparse": Real(1e-4, 1e+1, prior="log-uniform"),
      # },
      MultinomialNB(): {"alpha": Real(1e-10, 1e+1, prior="log-uniform")},
      AdaBoostClassifier(): {
          "n_estimators": Integer(25, 75),
          "learning_rate": Real(1e-6, 1e+1, prior="log-uniform"),
      },
      XGBClassifier(): {
          "learning_rate": Real(1e-6, 1e+1, prior="log-uniform"),
          "n_estimators": Integer(10, 100),
      },
      RandomForestClassifier(): { 
          "n_estimators": Integer(75, 200),
      },
  }
}
 
async def remove_all_flows():
  from prefect.client import get_client
  orion_client = get_client()
  flows = await orion_client.read_flows()
  for flow in flows:
    flow_id = flow.id
    print(f"Deleting flow: {flow.name}, {flow_id}")
    await orion_client._client.delete(f"/flows/{flow_id}")
    print(f"Flow with UUID {flow_id} deleted")

if clearHistory: await remove_all_flows()

In [7]:
from prefect import unmapped
from tqdm import tqdm

@task()
def filterTable(table, filterString):
    if not filterString: return table
    print(f"Filtering: {filterString}")
    filteredTable = table.query(filterString)
    return filteredTable

@task()
def applyAlleleModel(values, columns, genotypeIDs):
    # some genotype IDs are subset of column names (or vice versa)
    genotypeDict = dict()
    resolvedGenotypeIDs = set()
    for id in tqdm(genotypeIDs, unit='id'):
        for j, column in enumerate(columns):
            if id in column or column in id:
                # implement allele model
                genotypeDict[f"{column}"] = [(
                                np.sum([int(allele) for allele in genotype.replace("'", "").split("/")]) # split by allele delimiter 
                                if not config["vcfLike"]["binarize"] 
                                else np.clip(
                                        np.sum([int(allele) for allele in genotype.replace("'", "").split("/")]), 
                                        a_max=1, a_min=None)
                                )
                            if any(char.isdigit() for char in genotype)
                            else np.nan
                            for genotype in values[:,j]
                        ]
                columns = np.delete(columns, j)
                values = np.delete(values, j, axis=1)
                resolvedGenotypeIDs.update({id})
                break
    missingGenotypeIDs = set(genotypeIDs) - resolvedGenotypeIDs  # leftover columns are missing
    return genotypeDict, missingGenotypeIDs, resolvedGenotypeIDs

@task()
def load():
    clinicalData = pd.read_excel(config['clinicalTable']['path'], index_col=config['clinicalTable']['idColumn']
                                ).drop_duplicates(subset=config['clinicalTable']['uniqueIdColumn'])
    externalSamples = [pd.read_csv(path, sep='\t', index_col=idColumn) for path, idColumn in zip(config['externalTables']['path'], config['externalTables']['idColumn'])]
    annotatedVCF = pd.read_csv(
        config['vcfLike']['path'], sep='\t', dtype=str, index_col=config['vcfLike']['indexColumn'], 
        ) if "xlsx" not in config['vcfLike']['path'] else pd.read_excel(
            config['vcfLike']['path'], sheet_name=(config['vcfLike']['sheet'] if config['vcfLike']['sheet'] else None), 
            dtype=str, na_values=['.'], keep_default_na=False
        )
    # remove null chromosome positions
    annotatedVCF[config['vcfLike']['indexColumn']] = annotatedVCF[config['vcfLike']['indexColumn']].astype(str).replace('', np.nan)
    return clinicalData, externalSamples, annotatedVCF.dropna(subset=config['vcfLike']['indexColumn']).set_index(config['vcfLike']['indexColumn'])

@flow(task_runner=ConcurrentTaskRunner(), log_prints=True)
async def processInputFiles():
    clinicalData, externalSamples, annotatedVCF = load()
    
    filteredClinicalData = filterTable(clinicalData, config['clinicalTable']['filters'])
    print(f"filtered {len(clinicalData) - len(filteredClinicalData)} samples from clinical data")
    filteredExternalSamples = [filterTable(externalSampleTable, filterString) for externalSampleTable, filterString in zip(externalSamples, config['externalTables']['filters'])]
    for i, (externalSampleTable, path) in enumerate(zip(filteredExternalSamples, config['externalTables']['path'])):
        print(f"filtered {len(externalSamples[i]) - len(externalSampleTable)} samples from external data {path}")
    filteredVCF = filterTable(annotatedVCF, config['vcfLike']['filters'])
    print(f"filtered {annotatedVCF.shape[0] - filteredVCF.shape[0]} variants from VCF")
    
    caseIDsMask, controlIDsMask = [
        filteredClinicalData[config['clinicalTable']['labelColumn']].isin(labels).dropna()
        for labels in (config['clinicalTable']['caseLabels'], config['clinicalTable']['controlLabels'])]
    
    caseIDs = caseIDsMask[caseIDsMask].index.to_numpy()
    controlIDs = controlIDsMask[controlIDsMask].index.to_numpy()
    for i, label in enumerate(config['externalTables']['label']):
        if label == config['clinicalTable']['caseAlias']:
            caseIDs = np.append(caseIDs, filteredExternalSamples[i].index.to_numpy())
        elif label == config['clinicalTable']['controlAlias']:
            controlIDs = np.append(controlIDs, filteredExternalSamples[i].index.to_numpy())

    # cast genotypes as numeric, drop chromosome positions with missing values
    caseGenotypeFutures, controlGenotypeFutures = applyAlleleModel.map(
            unmapped(filteredVCF.to_numpy()),
            unmapped(filteredVCF.columns.to_numpy()), 
            genotypeIDs=[IDs for IDs in (caseIDs, controlIDs)]
        ) 
    caseGenotypeDict, missingCaseIDs, resolvedCaseIDs = caseGenotypeFutures.result()
    controlGenotypeDict, missingControlIDs, resolvedControlIDs = controlGenotypeFutures.result()
    
    if len(missingCaseIDs) > 0 or len(missingControlIDs) > 0:
        for alias, IDs in {"caseAlias": missingCaseIDs, "controlAlias": missingControlIDs}.items():
            print(f"\nmissing {len(IDs)} {config['clinicalTable'][alias]} IDs:\n {IDs}")
            
    caseGenotypes = pd.DataFrame.from_dict(caseGenotypeDict)
    caseGenotypes.index.name = filteredVCF.index.name
    caseGenotypes.index = filteredVCF.index
    controlGenotypes = pd.DataFrame.from_dict(controlGenotypeDict)
    controlGenotypes.index.name = filteredVCF.index.name
    controlGenotypes.index = filteredVCF.index
    
    caseIDs = resolvedCaseIDs
    controlIDs = resolvedControlIDs
    
    print(f"\n{len(caseIDs)} cases:\n {caseIDs}")
    print(f"\n{len(controlIDs)} controls:\n {controlIDs}")
    # filter allele frequencies
    allGenotypes = pd.concat([caseGenotypes.dropna(how='any', axis=0), controlGenotypes.dropna(how='any', axis=0)], axis=1)
    filteredGenotypes = allGenotypes.loc[
        allGenotypes.gt(0).sum(axis=1).divide(len(allGenotypes.columns)) >= config['vcfLike']['minAlleleFrequency']]
    print(f"Filtered {len(filteredVCF) - len(filteredGenotypes)} alleles with frequency below {'{:.3%}'.format(config['vcfLike']['minAlleleFrequency'])}")
    print(f"Kept {len(filteredGenotypes)} alleles")
    
    caseGenotypes = filteredGenotypes.loc[:,caseGenotypes.columns]
    controlGenotypes = filteredGenotypes.loc[:,controlGenotypes.columns]
    
    return [caseGenotypes, caseIDs, controlGenotypes, controlIDs, filteredClinicalData]

caseGenotypes, caseIDs, controlGenotypes, controlIDs, clinicalData = await processInputFiles()
print(f"\nclinical data:\n{clinicalData.head()}")

02:00:46.500 | INFO    | prefect.engine - Created flow run 'holistic-dragonfly' for flow 'processInputFiles'

02:00:46.620 | INFO    | Flow run 'holistic-dragonfly' - Created task run 'load-0' for task 'load'

02:00:46.621 | INFO    | Flow run 'holistic-dragonfly' - Executing 'load-0' immediately...

Unknown extension is not supported and will be removed


02:01:50.574 | INFO    | Task run 'load-0' - Finished in state Completed()

...

02:02:45.755 | INFO    | Flow run 'holistic-dragonfly' - 
684 controls:
 {'CGND-HDA-02444', 'HG00100', 'NA20758', 'NA11894', 'HG01536', 'HG01624', 'HG00122', 'CGND-HDA-01519', 'CGND-HDA-02724', 'NA20515', 'CGND-HDA-03585', 'NA20769', 'CGND-HDA-03468', 'HG02238', 'CGND-HDA-03797', 'HG01620', 'CGND-HDA-03978', 'NA20802', 'CGND-HDA-00564', 'CGND-HDA-02516', 'HG00145', 'CGND-HDA-01541', 'HG01670', 'NA20772', 'HG02221', 'CGND-HDA-01537', 'CGND-HDA-01486', 'HG01776', 'NA20808', 'CGND-HDA-01691', 'CGND-HDA-01653', 'CGND-HDA-03548', 'CGND-HDA-02764', 'CGND-HDA-03668', 'CGND-HDA-01700', 'CGND-HDA-02513', 'CGND-HDA-01593', 'CGND-HDA-01551', 'CGND-HDA-01651', 'NA12155', 'NA12812', 'CGND-HDA-03876', 'CGND-HDA-01692', 'CGND-HDA-03792', 'HG01705', 'CGND-HDA-01526', 'CGND-HDA-03773', 'CGND-HDA-01532', 'HG00143', 'CGND-HDA-03781', 'CGND-HDA-00209', 'NA12282', 'NA12760', 'CGND-HDA-01900', 'NA11843', 'HG01761', 'NA12763', 'CGND-HDA-02760', 'CGND-HDA-03098', 'CGND-HDA-02189', 'NA20756', 'CGND-HDA-03870', 'HG00255', 'NA20535', 'NA20832', 'NA11918', 'NA20544', 'NA20761', 'HG01617', 'CGND-HDA-03727', 'NA20754', 'CGND-HDA-03093', 'HG00110', 'NA12827', 'CGND-HDA-01502', 'HG01747', 'CGND-HDA-01453', 'CGND-HDA-03522', 'HG00160', 'NA20516', 'CGND-HDA-01663', 'HG02233', 'CGND-HDA-03421', 'CGND-HDA-03142', 'HG01613', 'HG01610', 'NA12044', 'HG01524', 'NA12843', 'HG00159', 'NA20529', 'CGND-HDA-01438', 'NA20753', 'CGND-HDA-03734', 'CGND-HDA-01655', 'NA11829', 'CGND-HDA-03470', 'CGND-HDA-03486', 'CGND-HDA-02021', 'CGND-HDA-01594', 'CGND-HDA-03089', 'HG00112', 'CGND-HDA-03785', 'CGND-HDA-03778', 'CGND-HDA-03874', 'HG01765', 'NA12813', 'CGND-HDA-03167', 'HG00258', 'CGND-HDA-02706', 'NA11920', 'NA20502', 'CGND-HDA-02746', 'HG01504', 'CGND-HDA-02716', 'HG00148', 'CGND-HDA-03481', 'CGND-HDA-02514', 'CGND-HDA-03739', 'HG01503', 'NA12718', 'NA20798', 'CGND-HDA-03201', 'CGND-HDA-02440', 'CGND-HDA-03770', 'HG01697', 'HG01784', 'NA12156', 'NA20771', 'CGND-HDA-01901', 'HG00259', 'NA12003', 'NA20815', 'HG01525', 'HG01531', 'CGND-HDA-03790', 'CGND-HDA-02831', 'CGND-HDA-03986', 'HG01501', 'CGND-HDA-03753', 'NA10847', 'NA12749', 'CGND-HDA-03671', 'NA11830', 'CGND-HDA-01559', 'NA20787', 'CGND-HDA-01818', 'HG00237', 'HG01608', 'HG02239', 'NA12873', 'HG00244', 'NA20827', 'CGND-HDA-02738', 'HG01757', 'CGND-HDA-01604', 'HG00109', 'NA10851', 'HG00243', 'HG00154', 'NA20543', 'HG01680', 'HG00158', 'CGND-HDA-03524', 'HG01602', 'CGND-HDA-03479', 'CGND-HDA-03977', 'HG01515', 'HG01786', 'HG00128', 'HG01682', 'HG01768', 'CGND-HDA-03074', 'CGND-HDA-03562', 'CGND-HDA-01410', 'HG01630', 'NA20775', 'HG00103', 'CGND-HDA-01607', 'CGND-HDA-03166', 'HG00114', 'CGND-HDA-01456', 'HG00251', 'CGND-HDA-02515', 'CGND-HDA-01853', 'NA12043', 'HG00151', 'HG02231', 'CGND-HDA-01538', 'CGND-HDA-02766', 'NA20509', 'HG01699', 'CGND-HDA-03782', 'HG00131', 'CGND-HDA-02519', 'HG00146', 'CGND-HDA-03079', 'HG01679', 'CGND-HDA-01448', 'CGND-HDA-02088', 'HG01789', 'NA12347', 'HG01628', 'HG00157', 'CGND-HDA-01543', 'HG00231', 'NA20513', 'CGND-HDA-03476', 'CGND-HDA-01605', 'HG01626', 'CGND-HDA-00260', 'CGND-HDA-03783', 'CGND-HDA-02734', 'CGND-HDA-03527', 'CGND-HDA-01664', 'NA12287', 'CGND-HDA-03197', 'CGND-HDA-01830', 'HG01673', 'NA20518', 'HG01615', 'HG01516', 'HG00123', 'CGND-HDA-01427', 'CGND-HDA-00565', 'HG00120', 'NA20520', 'CGND-HDA-03794', 'HG01675', 'NA06985', 'CGND-HDA-03774', 'NA20542', 'CGND-HDA-01417', 'HG02230', 'HG00140', 'NA20810', 'CGND-HDA-02833', 'HG02236', 'CGND-HDA-02223', 'CGND-HDA-01610', 'CGND-HDA-03951', 'HG01762', 'NA20505', 'HG00235', 'CGND-HDA-00434', 'CGND-HDA-03864', 'HG01669', 'HG01773', 'NA12286', 'CGND-HDA-02512', 'HG00245', 'CGND-HDA-01654', 'HG00257', 'HG00096', 'NA20796', 'CGND-HDA-01511', 'HG00116', 'CGND-HDA-03448', 'NA20534', 'CGND-HDA-04090', 'HG00254', 'HG01686', 'HG01708', 'HG00130', 'NA20587', 'CGND-HDA-02079', 'NA20826', 'CGND-HDA-02774', 'CGND-HDA-01841', 'CGND-HDA-01430', 'NA12400', 'CGND-HDA-01641', 'HG01678', 'HG00138', 'HG02232', 'NA12874', 'NA20812', 'CGND-HDA-02567', 

02:02:45.764 | INFO    | Flow run 'holistic-dragonfly' - Filtered 1426 alleles with frequency below 5.000%

02:02:45.765 | INFO    | Flow run 'holistic-dragonfly' - Kept 56 alleles

02:02:45.834 | INFO    | Flow run 'holistic-dragonfly' - Finished in state Completed()


clinical data:
                       Quote    Data File ID ExternalSubjectId   
ExternalSampleId                                                 
CGND-HDA-05557    CGND_14852  CGND-HDA-05557       NEUUF013XXL  \
CGND-HDA-05556    CGND_14852  CGND-HDA-05556       NEUHM496PGR   
CGND-HDA-05555    CGND_14852  CGND-HDA-05555       NEUPK599KHH   
CGND-HDA-05554    CGND_14852  CGND-HDA-05554       NEUHD589CVP   
CGND-HDA-05553    CGND_14852  CGND-HDA-05553       NEUXX223WT8   

                              Project     Site Sample Collected   
ExternalSampleId                                                  
CGND-HDA-05557    ALS Natural History  Henry Ford Health System  \
CGND-HDA-05556    ALS Natural History  Henry Ford Health System   
CGND-HDA-05555    ALS Natural History  Henry Ford Health System   
CGND-HDA-05554    ALS Natural History  Henry Ford Health System   
CGND-HDA-05553    ALS Natural History  Henry Ford Health System   

                   Site Specimen Collected     Sex 

## Evaluate model stack

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.calibration import CalibrationDisplay
from sklearn.metrics import RocCurveDisplay, roc_auc_score, auc
from sklearn.preprocessing import MinMaxScaler
from skopt.plots import plot_convergence

from skopt import BayesSearchCV

from fastnumbers import check_real

from types import SimpleNamespace
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from prefect.utilities.annotations import unmapped

import neptune
from neptune.types import File

import shap

from inspect import isclass
from io import StringIO
import traceback

# stop API errors when awaiting results
# !prefect config set PREFECT_RESULTS_PERSIST_BY_DEFAULT=True

def getFeatureImportances(model, data, featureLabels):
  """Get feature importances from fitted model and create SHAP explainer"""
  modelValuesDF = None
  if model.__class__.__name__ == "MultinomialNB":
      modelValuesDF = pd.DataFrame()
      for i, c in enumerate(model.feature_count_[0] if len(model.feature_count_.shape) > 1 else model.feature_count_):
          modelValuesDF.loc[i, f"log_prob_{config['clinicalTable']['controlAlias']}"] = model.feature_log_prob_[0][i]
          modelValuesDF.loc[i, f"log_prob_{config['clinicalTable']['caseAlias']}"] = model.feature_log_prob_[1][i]
  elif hasattr(model, "coef_"):
      modelValuesDF = pd.DataFrame()
      if len(model.coef_.shape) > 1:
        try:
          modelValuesDF[f"feature_importances_{config['clinicalTable']['controlAlias']}"] = model.coef_[0]
          modelValuesDF[f"feature_importances_{config['clinicalTable']['caseAlias']}"] = model.coef_[1]
        except IndexError:
          modelValuesDF[f"feature_importances_{config['clinicalTable']['caseAlias']}"] = model.coef_[0]                 
      else:
        modelValuesDF[f"feature_importances"] = model.coef_[0]
  elif hasattr(model, "feature_importances_"):
      modelValuesDF = pd.DataFrame()
      modelValuesDF[f"feature_importances_{config['clinicalTable']['caseAlias']}"] = model.feature_importances_

  if type(modelValuesDF) == pd.DataFrame:
      modelValuesDF.index = featureLabels
      
  # Cluster correlated and hierarchical features using masker
  masker = shap.maskers.Partition(data, clustering="correlation")
  
  shapExplainer = shap.explainers.Permutation(
    model.predict_proba if hasattr(model, "predict_proba") 
    else model.predict, 
    masker, 
    feature_names=["_".join(label)
      for label in featureLabels])
  shapValues = shapExplainer(data)
  return modelValuesDF, shapValues, shapExplainer


@task()
def plotCalibration(title, labelsPredictionsByInstance):
  # code from https://scikit-learn.org/stable/auto_examples/calibration/plot_calibration_curve.html
  fig, ax_calibration_curve = plt.subplots(figsize=(10, 10))
  colors = plt.cm.get_cmap("Dark2")

  calibration_displays = {}
  for i, (name, (labels, predictions)) in enumerate(labelsPredictionsByInstance.items()):
      display = CalibrationDisplay.from_predictions(
          [config["clinicalTable"]["caseAlias"] if label == 1 else label 
            for label in labels],
          predictions,
          pos_label=config["clinicalTable"]["caseAlias"],
          n_bins=10,
          name=name,
          ax=ax_calibration_curve,
          color=colors(i),
      )
      calibration_displays[name] = display

  ax_calibration_curve.grid()
  ax_calibration_curve.set_title(title)

  # Add histogram
  # grid_positions = [(i+2,j) for i in range(len(predictionsByModelName.keys())//2) for j in range(2)]
  # for i, modelName in enumerate(predictionsByModelName.keys()):
  #     row, col = grid_positions[i]
  #     ax = fig.add_subplot(gs[row, col])
  #     ax.hist(
  #         calibration_displays[modelName].y_prob,
  #         range=(0, 1),
  #         bins=10,
  #         label=modelName,
  #         color=colors(i),
  #     )
  #     ax.set(title=modelName, xlabel="Mean predicted probability", ylabel="Count")

  plt.tight_layout()
  return fig

@task()
def plotAUC(title, labelsPredictionsByInstance):
  # trace AUC for each set of predictions
  tprs = []
  aucs = []
  mean_fpr = np.linspace(0, 1, 100)

  fig, ax = plt.subplots(figsize=(10, 10))
  for name, (labels, predictions) in labelsPredictionsByInstance.items():
    # plot ROC curve for this fold
    viz = RocCurveDisplay.from_predictions([
                          config["clinicalTable"]["caseAlias"] if label == 1 else label 
                          for label in labels], 
                        predictions,
                        name=name,
                        pos_label=config["clinicalTable"]["caseAlias"],
                        alpha=0.6, lw=2, ax=ax)
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)
    
  # summarize ROCs per fold and plot standard deviation
  ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
      label='Chance', alpha=.8)
  mean_tpr = np.mean(tprs, axis=0)
  mean_tpr[-1] = 1.0
  mean_auc = auc(mean_fpr, mean_tpr)
  std_auc = np.std(aucs)
  ax.plot(mean_fpr, mean_tpr, color='b',
          label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
          lw=4, alpha=.8)
  std_tpr = np.std(tprs, axis=0)
  tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
  tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
  ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                  label=r'$\pm$ 1 std. dev.')

  ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
        title=title)
  ax.legend(loc="lower right")
  ax.set(title=title)
  return fig


@task()
def plotConfusionMatrix():
  pass

@task()
def plotSampleAccuracy():
  pass

@task()
def plotOptimizer(title, resultsByInstance):
  # code from https://scikit-learn.org/stable/auto_examples/calibration/plot_calibration_curve.html
  fig = plt.figure(figsize=(10, 10))
  gs = GridSpec(2, 2)
  colors = plt.cm.get_cmap("Dark2")
  ax_convergence = fig.add_subplot(gs[:2, :2])
  plot_convergence(*[
    (modelName, result) for modelName, result in resultsByInstance.items()], 
    ax=ax_convergence, color=colors)
  ax_convergence.set(title=title)
  plt.tight_layout()
  return fig

@task()
def prepareDatasets(caseGenotypes, controlGenotypes, verbose=True):
    caseIDs = caseGenotypes.columns
    controlIDs = controlGenotypes.columns
    # store number of cases & controls
    caseControlCounts = [len(caseIDs), len(controlIDs)]
    # determine which has more samples
    labeledIDs = [caseIDs, controlIDs]
    majorIDs = labeledIDs[np.argmax(caseControlCounts)]
    minorIDs = labeledIDs[np.argmin(caseControlCounts)]
    # downsample larger group to match smaller group
    majorIndex = np.random.choice(np.arange(len(majorIDs)), min(caseControlCounts), replace=False)
    
    excessMajorIDs, balancedMajorIDs = [], []
    for index, id in enumerate(majorIDs):
      if index in majorIndex:
        balancedMajorIDs.append(id)
      else:
        excessMajorIDs.append(id)
    
    allGenotypes = pd.concat([caseGenotypes, controlGenotypes], axis=1)
    
    genotypeExcessIDs, genotypeTrainIDs = [], []
    # match IDs between genotype and clinical data; dataframe labels have label suffixes
    unmatchedTrainIDs = balancedMajorIDs + minorIDs
    for label in tqdm(allGenotypes.columns, desc="Matching IDs", unit="ID"):
      for setType in ["excess", "train"]:
        idSet = excessMajorIDs if setType == "excess" else unmatchedTrainIDs
        for i, id in enumerate(idSet):
          if (id in label) or (label in id): 
            if setType == "train":
              if label not in genotypeTrainIDs: genotypeTrainIDs.append(label)
            elif setType == "excess":
              if label not in genotypeExcessIDs: genotypeExcessIDs.append(label)
            idSet = np.delete(idSet, i)
            break
          
    if verbose:
      print(f"\n{len(genotypeTrainIDs)} for training:\n{genotypeTrainIDs}")
      print(f"\n{len(genotypeExcessIDs)} are excess:\n{genotypeExcessIDs}")
      print(f"\nVariant count: {len(allGenotypes.index)}")
      
    samples = allGenotypes.loc[:, genotypeTrainIDs].dropna(how='any')  # drop variants with missing values
    excessMajorSamples = allGenotypes.loc[:, genotypeExcessIDs]

    variantIndex = list(samples.index)
    pass
    scaler = MinMaxScaler()
    embedding = {
        'sampleIndex': genotypeTrainIDs,
        'labels': np.array([1 if id in caseIDs else 0 for id in genotypeTrainIDs]),
        'samples': scaler.fit_transform(samples).transpose(), # samples are now rows (samples, variants)
        'excessMajorIndex': genotypeExcessIDs,
        'excessMajorLabels': [1 if id in caseIDs else 0 for id in genotypeExcessIDs],
        'excessMajorSamples': scaler.fit_transform(excessMajorSamples).transpose(),
        'variantIndex': variantIndex,
    }
    return embedding

@task()
def evaluate(trainIndices, testIndices, model, labels, samples, variantIndex, sampleIndex, parameterSpace, cvIterator):
    fittedOptimizer = optimizeHyperparameters(
        samples[trainIndices], labels[trainIndices], model, 
        parameterSpace, cvIterator, 'neg_mean_squared_error')
    model.set_params(**fittedOptimizer.best_params_)
    model.fit(samples[trainIndices], labels[trainIndices])
    # get model prediction probabilities
    try:
      probabilities = model.predict_proba(samples[testIndices])
    except AttributeError:
      probabilities = model.predict(samples[testIndices])
      if len(probabilities.shape) <= 1:
        probabilities = np.array([[1 - p, p] for p in probabilities])
    predictions = np.argmax(probabilities, axis=1)
    modelValues, shapValues, shapExplainer = getFeatureImportances(
      model, samples[testIndices], variantIndex)
    globalExplanations = modelValues
    localExplanations = shapValues
    trainLabels = np.array(labels[trainIndices])
    testLabels = np.array(labels[testIndices])
    trainIDs = np.array([sampleIndex[i] for i in trainIndices]) 
    testIDs = np.array([sampleIndex[i] for i in testIndices])
    return globalExplanations, localExplanations, probabilities, predictions, testLabels, trainLabels, trainIDs, testIDs, fittedOptimizer, shapExplainer

def optimizeHyperparameters(samples, labels, model, parameterSpace, cvIterator, metricFunction, n_jobs=1):
  # hyperparameter search (inner cross-validation)
  optimizer = BayesSearchCV(
    model, parameterSpace, cv=cvIterator, n_jobs=n_jobs, n_points=4,
    return_train_score=True, n_iter=100, scoring=metricFunction)
  # train / optimize parameters
  optimizer.fit(samples, labels)
  return optimizer

def serializeDataFrame(dataframe):
  stream = StringIO()
  dataframe.to_csv(stream)
  return File.from_stream(stream, extension='csv')

@flow(
  task_runner=RayTaskRunner(
    init_kwargs={'address': parallelRunner.address_info['address'], 'configure_logging': True, 'logging_level': logging.WARN}
  ))
async def classify():
  # caseGenotypes, caseIDs, controlGenotypes, controlIDs, clinicalData = await processInputFiles()
  outerCvIterator = StratifiedKFold(n_splits=config['sampling']['crossValIterations'], shuffle=False)
  innerCvIterator = outerCvIterator
  results = {}
  projectTracker = neptune.init_project(project=f'{config["tracking"]["entity"]}/{config["tracking"]["project"]}', api_token=config['tracking']['token'])
  for i in tqdm(range(config['sampling']['bootstrapIterations']), unit='cohort'):
    embedding = prepareDatasets(caseGenotypes, controlGenotypes, verbose=(True if i == 0 else False))
    deserializedIDs = list()
    for id in embedding['sampleIndex']:
      deserializedIDs.extend(id.split("__"))
    totalSampleCount = len(embedding['samples'])
    caseCount = np.count_nonzero(embedding['labels'])
    print(f"{totalSampleCount} samples\n")
    print(f"{caseCount} cases\n")
    print(f"{totalSampleCount - caseCount} controls\n")
    results[i] = {}
    results['samples'] = {}
    results['labels'] = {}
    for j, (model, parameterSpace) in enumerate(config['modelStack'].items()):
      if model.__class__.__name__ not in results:
        results[i][model.__class__.__name__] = {"predictions": [], "optimizeResults": []} 
      current = {}
      # check if model is initialized
      if isclass(model):
        if model.__name__ == 'TabNetClassifier':
          model = model(verbose=False, optimizer_fn=Lion)
      print(f"Iteration {i+1} with model {model.__class__.__name__}")
      runTracker = neptune.init_run(project=f'{config["tracking"]["entity"]}/{config["tracking"]["project"]}', api_token=config['tracking']['token'])
      runTracker['sys/tags'].add(model.__class__.__name__)
      runTracker['bootstrapIteration'] = i+1
      runTracker["config"] = {key: (item if check_real(item) or isinstance(item, str) else str(item)) for key, item in config.items()}
      runTracker['embedding'].upload(serializeDataFrame(pd.DataFrame(data=embedding["samples"], columns=embedding["variantIndex"], index=embedding["sampleIndex"])))
      runTracker['clinicalData'].upload(serializeDataFrame(clinicalData.loc[clinicalData.index.isin(deserializedIDs)]))
      # outer cross-validation
      crossValIndices = np.array([
          (cvTrainIndices, cvTestIndices) 
          for (cvTrainIndices, cvTestIndices) in outerCvIterator.split(embedding['samples'], embedding['labels'])])
      current["trainIndices"] = crossValIndices[:,0]
      current["testIndices"] = crossValIndices[:,1]
      # this could be parallelized more efficiently since Shapely explanations are independent
      outerCrossValFutures = evaluate.map(
          trainIndices=crossValIndices[:,0], 
          testIndices=crossValIndices[:,1],
          model=unmapped(model), 
          labels=unmapped(embedding['labels']), 
          samples=unmapped(embedding['samples']), 
          variantIndex=unmapped(embedding['variantIndex']), 
          sampleIndex=unmapped(embedding['sampleIndex']), 
          parameterSpace=unmapped(parameterSpace), 
          cvIterator=unmapped(innerCvIterator))
      outerCrossValResults = zip(*[fold.result() for fold in outerCrossValFutures])
      resultNames = ["globalExplanations", "localExplanations", "probabilities", "predictions", "testLabels", "trainLabels", "trainIDs", "testIDs", "fittedOptimizers", "shapExplainers"]
      current = {**current, **{name: result for name, result in zip(resultNames, outerCrossValResults)}}
      
      # plot AUC & elbow
      runTracker["modelParams"] = {k+1: current["fittedOptimizers"][k].best_params_ for k in range(config['sampling']['crossValIterations'])}
      plotSubtitle = f"""
            {config["tracking"]["name"]}, {embedding["samples"].shape[1]} variants
            Minor allele frequency over {'{:.1%}'.format(config['vcfLike']['minAlleleFrequency'])}
            {np.count_nonzero(embedding['labels'])} {config["clinicalTable"]["caseAlias"]}s, {len(embedding['labels']) - np.count_nonzero(embedding['labels'])} {config["clinicalTable"]["controlAlias"]}s"""
      runTracker['plots/aucPlot'] = plotAUC(f"""
        Receiver Operating Characteristic (ROC) Curve
        {model.__class__.__name__} with {config['sampling']['crossValIterations']}-fold cross-validation
        {plotSubtitle}
        """,
        {f"Fold {k+1}": (current["testLabels"][k], np.array(current["probabilities"][k])[:,1])
         if len(current["probabilities"][k][0].shape) >= 1 
         else (current["testLabels"][k], current["probabilities"][k]) 
         for k in range(config['sampling']['crossValIterations'])},
        )
      runTracker['plots/convergencePlot'] = plotOptimizer(f"""
        Hyperparameter convergence, mean squared error
        {model.__class__.__name__} with {config['sampling']['crossValIterations']}-fold cross-validation
        {plotSubtitle}
        """,
        {f"Fold {k+1}": [result for result in current["fittedOptimizers"][k].optimizer_results_]
         for k in range(config['sampling']['crossValIterations'])},
        )
      
      # update model metrics
      current["testAUC"] = [roc_auc_score(labels, (probabilities[:,1] if len(probabilities.shape) > 1 else probabilities))
        for labels, probabilities in zip(current["testLabels"], current["probabilities"])]
      runTracker["meanAUC"] = np.mean(current["testAUC"])
      # update sample metrics
      sampleResults = {}
      for fold in range(config['sampling']['crossValIterations']):
        for j, sampleID in enumerate(current['testIDs'][fold]):
          try:
            results['samples'][sampleID] += current["probabilities"][fold][j]
          except KeyError:
            results['samples'][sampleID] = [current["probabilities"][fold][j]]
          finally:
            results['labels'][sampleID] = current["testLabels"][fold][j]
      results[i][model.__class__.__name__] = current
      
      runTracker['shapExplanationsPerFold'].upload(File.as_pickle(current["localExplanations"]))
      runTracker['shapExplainersPerFold'].upload(File.as_pickle(current["shapExplainers"]))
      runTracker['trainIDs'].upload(File.as_pickle(current["trainIDs"]))
      runTracker['testIDs'].upload(File.as_pickle(current["testIDs"]))
      runTracker['testLabels'].upload(File.as_pickle(current["testLabels"]))
      runTracker['trainLabels'].upload(File.as_pickle(current["trainLabels"]))
      
      # plot feature importance
      for j in range(config["sampling"]["crossValIterations"]):
        if current["globalExplanations"][j] is not None:
          runTracker[f"globalFeatureImportance/{j+1}"].upload(serializeDataFrame(current["globalExplanations"][j]))
        localExplanations = current["localExplanations"][j]
        caseExplanations = localExplanations
        caseExplanations.values = caseExplanations.values[:,:,1] if len(caseExplanations.values.shape) > 2 else caseExplanations.values
        heatmap = plt.figure()
        plt.title(f"""
          Shapely explanations from {model.__class__.__name__}
          Fold {j+1}
          {plotSubtitle}
          """) 
        shap.plots.heatmap(caseExplanations, show=False)
        runTracker[f"plots/featureHeatmap/{j+1}"] = heatmap
        plt.close(heatmap)
        labelsProbabilities = ((current["testLabels"][j], np.array(current["probabilities"][j])[:,1])
          if len(current["probabilities"][j][0].shape) >= 1 
          else (current["testLabels"][j], current["probabilities"][j]))
        stdDeviation = np.std((labelsProbabilities[1] - labelsProbabilities[0])**2)
        for k in range(len(current["testIDs"][j])):
          probability = labelsProbabilities[1][k] if isinstance(labelsProbabilities[1][k], np.ndarray) else labelsProbabilities[1][k]
          label = labelsProbabilities[0][k] if isinstance(labelsProbabilities[0][k], np.ndarray) else labelsProbabilities[0][k]
          if config['tracking']['plotAllSampleImportances'] or np.absolute((probability - label)**2) <= stdDeviation:
            sampleID=current['testIDs'][j][k]
            waterfallPlot = plt.figure()
            plt.title(f"""
              {sampleID}
              Shapely explanations from {model.__class__.__name__}
              Fold {j+1}
              {plotSubtitle}
              """)
            # patch parameter bug: https://github.com/slundberg/shap/issues/2362
            to_pass = SimpleNamespace(**{
                            'values': localExplanations[k].values,
                            'data': localExplanations[k].data,
                            'display_data': None,
                            'feature_names': localExplanations.feature_names,
                            'base_values': localExplanations[k].base_values[current['testLabels'][j][k]] if len(localExplanations[k].base_values.shape) == 1 else localExplanations[k].base_values, 
              })
            shap.plots.waterfall(to_pass, show=False)
            try:
              runTracker[f"plots/samples/{j+1}/{sampleID}"] = waterfallPlot
            except Exception as e:
              runTracker[f"plots/samples/{j+1}/{sampleID}"] = f"""failed to plot: {traceback.format_exc()}"""
            plt.close(waterfallPlot)

      runTracker["sampleResults"].upload(serializeDataFrame(pd.DataFrame.from_dict({
        "probability": [probability[1] for foldResults in current["probabilities"] for probability in foldResults], 
        "id": [id for foldResults in current["testIDs"] for id in foldResults]}, dtype=object).set_index("id")))
      plt.close('all')
      runTracker.stop()
  
  labelsProbabilitiesByModelName = dict()
  for modelName in results[0].keys():
    labelsProbabilitiesByModelName[modelName] = [[], []]
    for k in range(config['sampling']['bootstrapIterations']):
      # append labels
      labelsProbabilitiesByModelName[modelName][0] = np.hstack(
        [labelsProbabilitiesByModelName[modelName][0], 
        np.concatenate(results[k][modelName]["testLabels"])])
      # append probabilities
      labelsProbabilitiesByModelName[modelName][1] = np.hstack(
        [labelsProbabilitiesByModelName[modelName][1], 
        np.concatenate(results[k][modelName]["probabilities"])[:,1] 
          if len(results[k][modelName]["probabilities"][0].shape) >= 1 
          else np.concatenate(results[k][modelName]["probabilities"])])
      
  seenCaseCount, seenControlCount = 0, 0
  sampleResults = []
  
  for sampleID in results['samples'].keys():
    flattenedProbabilities = np.array([
      result[1] if len(caseExplanations.values.shape) > 2 
      else result 
      for foldResult in results['samples'][sampleID] 
      for result in foldResult]) 
    sampleResults += [{
      "probability": results['samples'][sampleID][:1],
      "accuracy": np.mean([np.ceil(caseProbability) == results['labels'][sampleID] 
                            for caseProbability in flattenedProbabilities]),
      "id": sampleID,
    }]
    if results["labels"][sampleID] == 1:
      seenCaseCount += 1
    else:
      seenControlCount += 1
      
  plotSubtitle = f"""{config['sampling']['crossValIterations']}x cross-validation over {config['sampling']['bootstrapIterations']} bootstrap iterations
  {config["tracking"]["name"]}, {embedding["samples"].shape[1]} variants
  Minor allele frequency over {'{:.1%}'.format(config['vcfLike']['minAlleleFrequency'])}
  {seenCaseCount} cases, {seenControlCount} controls
  """
  # print(f'preds 1: {results[0]["TabNetClassifier"]["predictions"]}\n')
  # print("labels:")
  # projectTracker["sampleResults"].upload(serializeDataFrame(pd.DataFrame.from_records(sampleResults, index="id")))
  projectTracker["aucPlot"].upload(plotAUC(f"""
    Receiver Operating Characteristic (ROC) Curve
    {plotSubtitle}
    """,
    labelsProbabilitiesByModelName,
    ))
  
  projectTracker["calibrationPlot"].upload(File.as_image(plotCalibration(f"""
    Calibration Curve
    {plotSubtitle}
    """,
    labelsProbabilitiesByModelName,
    )))
    
  projectTracker["convergencePlot"].upload(File.as_image(plotOptimizer(f"""
    Convergence Plot
    {plotSubtitle}
    """,
    {modelName: [result
                for k in range(config['sampling']['bootstrapIterations'])
                for foldOptimizer in results[k][modelName]["fittedOptimizers"]
                for result in foldOptimizer.optimizer_results_
                ] 
      for modelName in results[0].keys() if modelName != "testLabels"
    })))
  projectTracker.stop()
  return results

results = await classify()